# Setup
## Imports

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

In [ ]:
import constants
import config

In [ ]:
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

## Config

In [ ]:
base_path = "sweeps/"

In [ ]:
open_loop_color = "forestgreen"
vision_and_force_color = "darkorchid"
vision_only_color = "dodgerblue"
force_only_color = "red"

In [ ]:
success_angle = 0.99*np.pi

## Functions

In [ ]:
def draw_trajs(datas, idxs=None, plot_manipulator=True, progs=None, num_times=5,
              fig_height=2.1, fig_width=6, cb_shrink=0.8, max_times=None):
    if idxs is None:
        sim_args = datas["sim_args"][0]
        datas = datas["out_data"]
    else:
        sim_args = datas["sim_args"][0]
        datas = [datas["out_data"][idx] for idx in idxs]
    w_L = sim_args['sim_params'].w_L
    h_L = sim_args['sim_params'].h_L
    r = sim_args['sim_params'].r
    if max_times is None:
        max_times = np.ones(len(datas))

    # Set to be arbitrary large/small numbers
    # IMPORTANT: x and y here refer to PLOT x and ys, not sim
    min_x = 10**9
    min_y = 10**9
    max_x = -10**9
    max_y = -10**9

    # Initialize plotting vars
    fig, axs = plt.subplots(1, len(datas), figsize=(fig_width,fig_height), constrained_layout=True, sharex=True, sharey=True)
    if len(datas) == 1:
        axs = [axs]
    axs[0].set_ylabel("$z$ position")
    cmap = plt.get_cmap("viridis_r")
    
    for i, data_, in enumerate(datas):
        start_idx = data_["any_links_in_contact_idx"]
        start_time = data_["times"][start_idx]
        end_time = data_["times"][-1]
        time_range = end_time - start_time

        if progs is None:
            times_ = np.linspace(start_time, end_time, num_times)
        else:
            if type(progs[0]) is list:
                progs_ = np.array(progs[i])
            else:
                progs_ = np.array(progs)
            times_ = progs_*time_range + start_time
        ax = axs[i]

        # Draw rectangles over links
        for t in times_:
            c = cmap(((t-start_time)/time_range)/max_times[i])
            idx = np.argmax(data_["times"] >= t)

            for x, y, ang in zip(
                    data_["horizontal_paper_traces"][:,idx],
                    data_["vertical_paper_traces"][:,idx],
                    data_["angle_paper_traces"][:,idx]):

                x0 = x - np.cos(ang)*w_L/2
                y0 = y - np.sin(ang)*w_L/2

                rect = plt. Rectangle((x0, y0), w_L, h_L,
                    angle=ang*180/np.pi, color=c,)
                ax.add_patch(rect)

                # Compute bounds to be used
                min_x = min(x - w_L*1.1*0.5, min_x)
                min_y = min(y - w_L*1.1*0.5, min_y)
                max_x = max(x + w_L*1.1*0.5, max_x)
                max_y = max(y + w_L*1.1*0.5, max_y)

        # Add pedestal
        ## Get first pedestal location
        rect = plt.Rectangle(
            (data_["horizontal_paper_traces"][0,0]-w_L/2,
             data_["vertical_paper_traces"][0,0]-h_L/2),
            10,
            10,
            angle=-90,
            color=(0.2, 0.2, 0.2, 1.0)
        )
        ax.add_patch(rect)

        # Add manipulator
        if plot_manipulator:
            for t in times_:
                c = cmap(((t-start_time)/time_range)/max_times[i])
                idx = np.argmax(data_["times"] >= t)

                x = data_["horizontal_manipulator_trace"][idx]
                y = data_["vertical_manipulator_trace"][idx]

                min_x = min(x - r*1.1, min_x)
                min_y = min(y - r*1.1, min_y)
                max_x = max(x + r*1.1, max_x)
                max_y = max(y + r*1.1, max_y)

                circ = plt.Circle((x, y), r*1.1, color=c, alpha=0.3, snap=False)
                ax.add_patch(circ)

        ax.set_xlabel("$x$ position")
        ax.set_aspect("equal")

    # Set lims (can do at the end because sharex and sharey are true)
    min_plot_x = min_x - (max_x-min_x)*0.1
    max_plot_x = max_x + (max_x-min_x)*0.1
    min_x_tick = np.floor(min_plot_x*10)/10
    max_x_tick = np.ceil(max_plot_x*10)/10
    xticks = np.arange(min_x_tick, max_x_tick+1e-5, 0.1)
    ax.set_xticks(xticks)

    min_plot_y = min_y - (max_y-min_y)*0.1
    max_plot_y = max_y + (max_y-min_y)*0.1
    min_y_tick = np.floor(min_plot_y*10)/10
    max_y_tick = np.ceil(max_plot_y*10)/10
    yticks = np.arange(min_y_tick, max_y_tick+1e-5, 0.1)
    ax.set_yticks(yticks)
    ax.set_ylim(min_plot_y, max_plot_y)
    plt.xlim(min_plot_x, max_plot_x)
    cb = fig.colorbar(cm.ScalarMappable(cmap="viridis_r"), shrink=cb_shrink)
    cb.set_label("Normalized time")

    return fig, axs, cb

In [ ]:
def get_overall_thetas(data, idx):
    p_W_LL = np.array([
            data["out_data"][idx]["horizontal_paper_traces"][-1,:],
            data["out_data"][idx]["vertical_paper_traces"][-1,:]
        ])
    p_W_FJ = np.array([
        data["out_data"][idx]["horizontal_paper_traces"][0,:] - data["sim_args"][idx]["sim_params"].w_L/2,
        data["out_data"][idx]["vertical_paper_traces"][0,:] + data["sim_args"][idx]["sim_params"].h_L*1.5
    ])
    p_FJ_LL = p_W_LL - p_W_FJ
    atan_x = -p_FJ_LL[0].flatten()
    atan_y = p_FJ_LL[1].flatten()
    overall_theta = np.arctan2(atan_y,atan_x)
    for i in range(1, len(overall_theta)):
        if overall_theta[i] - overall_theta[i-1] > 0.99*2*np.pi:
            overall_theta[i] -= 2*np.pi
        if overall_theta[i] - overall_theta[i-1] < -0.99*2*np.pi:
            overall_theta[i] += 2*np.pi
    return overall_theta

# Intro sweeps

In [ ]:
data = np.load(base_path + "traces.npz", allow_pickle=True)

In [ ]:
plot_manipulator=True
data = np.load(base_path + "traces.npz", allow_pickle=True)
sim_args = data["sim_args"][0]

fig, axs, cb = draw_trajs(data, plot_manipulator=plot_manipulator,
               fig_height=3)
titles = ["open loop setpoint\ntrajectory",
          r"Link feedback with" + "\n" +
          "constant $F_{Nff}$",
          "Link feedback with" + "\n" +
          "$F_{Nff}$ based on" + "\n" +
          "force measurements"]
for title, ax in zip(titles, axs):
    ax.set_title(title)
plt.suptitle("System behavior with various strategies")
fig.savefig("traj_drawings.pgf", backend="pgf")
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True)
colors = [open_loop_color, vision_only_color, vision_and_force_color]
titles = ["open loop setpoint trajectory",
          r"Link feedback with constant $F_{Nff}$",
          "Link feedback with $F_{Nff}$ based on force measurements"]
for i, (data_, color, title) in enumerate(zip(data['out_data'], colors, titles)):
    start_idx = data_["any_links_in_contact_idx"]
    plt.plot(data_["horizontal_paper_traces"][-1,start_idx:],
             data_["vertical_paper_traces"][-1,start_idx:], color=color, label=title)
    plt.plot(data_["horizontal_paper_traces"][-1,start_idx],
             data_["vertical_paper_traces"][-1,start_idx],
             marker='o', color=color)
    plt.plot(data_["horizontal_paper_traces"][-1,-1],
             data_["vertical_paper_traces"][-1,-1], marker='X', color=color)
plt.legend(frameon=False)
plt.xlabel("$y$ position")
plt.ylabel("$z$ position")
plt.gca().set_aspect("equal")

fig.savefig("traj_plots.pgf", backend="pgf")
plt.show()

# Task parameters

## Stiffness

In [ ]:
datas = [
    np.load(base_path + "vision_and_force__k_J.npz", allow_pickle=True),
    np.load(base_path + "open_loop__k_J.npz", allow_pickle=True),
    np.load(base_path + "vision_only__k_J.npz", allow_pickle=True),
    np.load(base_path + "force_only__k_J.npz", allow_pickle=True),
]
labels = ["Vision and force", "Open loop", "Vision only", "Force only"]
colors = [vision_and_force_color, open_loop_color, vision_only_color, force_only_color]

for i, (data, label, color) in enumerate(zip(datas, labels, colors)):
    # if i != 2:
    #     continue
    x_axis = data["sweep_vars"][:,0]
    y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])/success_angle
    succ_idxs = data["successes"].flatten()
    fail_idxs = np.logical_not(succ_idxs)
    plt.plot(x_axis, y_axis, linestyle='-', linewidth=2, label=label, zorder=i, color=color)
    plt.scatter(x_axis[succ_idxs], y_axis[succ_idxs], color=color, zorder=i)
    plt.scatter(x_axis[fail_idxs], y_axis[fail_idxs], facecolors='none', color=color, zorder=i)
plt.legend(frameon=False)
plt.xlabel("Hinge joint stiffness (Nm/rad)")
plt.ylabel(r"Progress ($\alpha_{max}/\pi$)")
plt.autoscale(enable=True, axis='x', tight=True)
plt.axhline(1, color='k', linestyle='--')
plt.title("Task progress vs. hinge joint stiffness ($k_J$)")
plt.gcf().savefig("k_J_sweep.pgf", backend="pgf")
plt.show()

In [ ]:
datas = [
    np.load(base_path + "vision_and_force__k_J.npz", allow_pickle=True),
    # np.load(base_path + "open_loop__k_J.npz", allow_pickle=True),
    # np.load(base_path + "vision_only__k_J.npz", allow_pickle=True),
    np.load(base_path + "force_only__k_J.npz", allow_pickle=True),
]
labels = [
    "Vision and force",
    # "Open loop",
    # "Vision only",
    "Force only"
]
colors = [
    vision_and_force_color,
    # open_loop_color,
    # vision_only_color,
    force_only_color
]
fig, axs = plt.subplots(len(datas), 1, figsize=(6,5), constrained_layout=True,sharex=True,sharey=True)
if len(datas) == 1:
    axs = [axs]

for i, (data, label, color, ax) in enumerate(zip(datas, labels, colors, axs)):
    x_axis = data["sweep_vars"][:,0]
    y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])/success_angle
    succ_idxs = data["successes"].flatten()
    fail_idxs = np.logical_not(succ_idxs)
    ax.plot(x_axis, y_axis, linestyle='-', linewidth=2, label=label, zorder=i, color=color)
    ax.scatter(x_axis[succ_idxs], y_axis[succ_idxs], color=color, zorder=i)
    ax.scatter(x_axis[fail_idxs], y_axis[fail_idxs], facecolors='none', color=color, zorder=i)
    ax.axhline(1, color='k', linestyle='--')
    ax.autoscale(enable=True, axis='x', tight=True)
    ax.set_title(label)
    ax.set_ylabel(r"Progress ($\alpha_{max}/\pi$)")
ax.set_xlabel("Hinge joint stiffness (Nm/rad)")
plt.ylim(0.85,1.05)
plt.suptitle("Task progress vs. hinge joint stiffness ($k_J$)")
plt.gcf().savefig("k_J_sweep_zoom.pgf", backend="pgf")
plt.show()

In [ ]:
plot_manipulator=True
data = np.load(base_path + "vision_only__k_J.npz", allow_pickle=True)
sim_args = data["sim_args"][0]

fig, axs, cb = draw_trajs(data, idxs=[-1], plot_manipulator=plot_manipulator,
                          progs=[0, 0.05, 0.25, 0.45, 0.7],
                          max_times=[0.7], fig_width=2.7, cb_shrink=1)
axs[0].set_title("$k_J = {}$ Nm/rad".format(data["sweep_vars"][-1,0]))
plt.suptitle("Hinge trajectory with visual feedback")
fig.savefig("k_J_vis_full_hinge.pgf", backend="pgf")
plt.show()

In [ ]:
plot_manipulator=True
data = np.load(base_path + "force_only__k_J.npz", allow_pickle=True)
sim_args = data["sim_args"][0]

idxs = [10, 17]
fig, axs, cb = draw_trajs(data, idxs=idxs, plot_manipulator=plot_manipulator, max_times=[1, 0.6],
                          progs=[[0,0.2,0.4,0.6,0.8,1], [0, 0.2, 0.4, 0.6]],
                          fig_width=4, cb_shrink=1)
titles = ["$k_J = {}$ Nm/rad".format(data["sweep_vars"][i,0]) for i in idxs]
for title, ax in zip(titles, axs):
    ax.set_title(title)
plt.suptitle(r"Hinge trajectories with force feedback")
fig.savefig("k_J_force_full_hinge.pgf", backend="pgf")
plt.show()

In [ ]:
data = np.load(base_path + "open_loop__mu.npz", allow_pickle=True)
fig, axs = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True)

color_map = cm.ScalarMappable(cmap="viridis_r", norm=plt.Normalize(0,np.max(data['sweep_vars'])))
for i, (data_, k_J) in enumerate(zip(data['out_data'][3:], data['sweep_vars'][3:,0])):
    c = color_map.to_rgba(k_J)
    start_idx = data_["any_links_in_contact_idx"]
    p = plt.plot(data_["horizontal_paper_traces"][-1,start_idx:],
             data_["vertical_paper_traces"][-1,start_idx:], color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,start_idx],
             data_["vertical_paper_traces"][-1,start_idx],
             marker='o', color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,-1],
             data_["vertical_paper_traces"][-1,-1], marker='X', color=c)
plt.xlabel("$y$ position")
plt.ylabel("$z$ position")

cb = fig.colorbar(color_map)
plt.gca().set_aspect("equal")
cb.set_label("Link stiffness (Nm/rad)")
plt.gcf().savefig("k_J_plot.pgf", backend="pgf")
plt.show()

## Link mass

### Sweep

In [ ]:
datas = [
    np.load(base_path + "vision_and_force__m_L.npz", allow_pickle=True),
    np.load(base_path + "open_loop__m_L.npz", allow_pickle=True),
    np.load(base_path + "vision_only__m_L.npz", allow_pickle=True),
    np.load(base_path + "force_only__m_L.npz", allow_pickle=True),
]
labels = [
    "Vision and force",
    "Open loop",
    "Vision only",
    "Force only"
]
colors = [
    vision_and_force_color,
    open_loop_color,
    vision_only_color,
    force_only_color
]
fig, axs = plt.subplots(len(datas), 1, figsize=(6,5), constrained_layout=True,sharex=True,sharey=True)
# fig, axs = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True,sharex=True,sharey=True)
if type(axs) != np.ndarray:
    axs = [axs, axs, axs, axs]

for i, (data, label, color, ax) in enumerate(zip(datas, labels, colors, axs)):
    ax.axhline(1, color='k', linestyle='--')
    x_axis = data["sweep_vars"]
    y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])/success_angle
    succ_idxs = data["successes"].flatten()
    fail_idxs = np.logical_not(succ_idxs)
    ax.plot(x_axis, y_axis, '-', label=label, zorder=i, color=color)
    ax.scatter(x_axis[succ_idxs], y_axis[succ_idxs], color=color, zorder=i)
    ax.scatter(x_axis[fail_idxs], y_axis[fail_idxs], facecolors='none', color=color, zorder=i)
    ax.set_ylabel("Progress\n" + r"($\alpha_{max}/\pi$)")
    ax.autoscale(enable=True, axis='x', tight=True)
    ax.set_title(label)
# plt.legend(frameon=False)
plt.xlabel("Link mass (kg)")
plt.suptitle("Task progress vs. link mass ($m_L$)")

plt.gcf().savefig("m_L_sweep.pgf", backend="pgf")
plt.show()

In [ ]:
plot_manipulator=True
data = np.load(base_path + "vision_only__m_L.npz", allow_pickle=True)
sim_args = data["sim_args"][0]

fig, axs, cb = draw_trajs(data, idxs=[-1], plot_manipulator=plot_manipulator,
                          fig_width=2.7, cb_shrink=1)
axs[0].set_title("$m_L = {}$ kg".format(data["sweep_vars"][-1,0]))
plt.suptitle("Hinge trajectory\nwith visual feedback")
fig.savefig("m_L_vis_full_hinge.pgf", backend="pgf")
plt.show()

In [ ]:
plot_manipulator=True
data = np.load(base_path + "force_only__m_L.npz", allow_pickle=True)
sim_args = data["sim_args"][0]

fig, axs, cb = draw_trajs(data, idxs=[-1], plot_manipulator=plot_manipulator,
                          fig_width=2.3, cb_shrink=1, progs=[0, 0.15, 0.3], max_times=[0.3])
axs[0].set_title("$m_L = {}$ kg".format(data["sweep_vars"][-1,0]))
plt.suptitle("Hinge trajectory\nwith force feedback")
fig.savefig("m_L_force_full_hinge.pgf", backend="pgf")
plt.show()

## Coefficient of friction

In [ ]:
datas = [
    np.load(base_path + "vision_and_force__mu.npz", allow_pickle=True),
    np.load(base_path + "open_loop__mu.npz", allow_pickle=True),
    np.load(base_path + "vision_only__mu.npz", allow_pickle=True),
    np.load(base_path + "force_only__mu.npz", allow_pickle=True),
]
labels = [
    "Vision and force",
    "Open loop",
    "Vision only",
    "Force only"
]
colors = [
    vision_and_force_color,
    open_loop_color,
    vision_only_color,
    force_only_color
]
plt.axhline(1, color='k', linestyle='--')
idxs = np.ones(16, dtype=bool)
idxs[0] = False
idxs[2] = False
idxs[3] = False
idxs[5] = False
idxs[7] = False
idxs[9] = False
idxs[11] = False
for i, (data, label, color) in enumerate(zip(datas, labels, colors)):
    x_axis = data["sweep_vars"]
    y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])/np.pi
    succ_idxs = data["successes"].flatten()
    fail_idxs = np.logical_not(succ_idxs)
    p = plt.plot(x_axis[idxs], y_axis[idxs], '-', label=label, zorder=i, color=color)
    plt.scatter(x_axis[np.logical_and(succ_idxs,idxs)],
                y_axis[np.logical_and(succ_idxs,idxs)], color=p[0].get_color(), zorder=i)
    plt.scatter(x_axis[np.logical_and(fail_idxs,idxs)],
                y_axis[np.logical_and(fail_idxs,idxs)], facecolors='none', color=p[0].get_color(), zorder=i)
plt.legend(frameon=False)
plt.autoscale(enable=True, axis='x', tight=True)
plt.xlabel("Coefficient of friction")
plt.ylabel("Progress" + r"($\alpha_{max}/\pi$)")
# ylim = plt.ylim()
plt.xlim(0,1)
plt.suptitle("Task progress vs. coefficient of friction ($\mu$)")
# plt.ylim(0,1.05)
plt.gcf().savefig("mu_sweep.pgf", backend="pgf")
plt.show()

In [ ]:
plot_manipulator=True
data = np.load(base_path + "force_only__mu.npz", allow_pickle=True)
sim_args = data["sim_args"][0]

idxs = [1, 8, -1]
fig, axs, cb = draw_trajs(data, idxs=idxs, plot_manipulator=plot_manipulator,
                          fig_height=2.6, cb_shrink=1)#, progs=[0, 0.15, 0.3], max_times=[0.3])
for ax, i in zip(axs, idxs):
    ax.set_title(r"$\mu = {:.2f}$".format(data["sweep_vars"][i,0]+0.005)) # fixing floating point rounding annoyance
plt.suptitle("Hinge trajectory with force feedback")
fig.savefig("mu_force_full_hinge.pgf", backend="pgf")
plt.show()

Currently cherry picking points because I think the variation at low coefficients of friction actually has to do with the different strategies making contact at different points.

# Control parameters

## Feedforward $F_N$

In [ ]:
datas = [
    np.load(base_path + "vision_and_force__ff_FN.npz", allow_pickle=True),
    np.load(base_path + "vision_only__ff_FN.npz", allow_pickle=True),
    np.load(base_path + "force_only__ff_FN.npz", allow_pickle=True),
]
labels = [
    "Vision and force",
    "Vision only",
    "Force only"
]
colors = [
    vision_and_force_color,
    vision_only_color,
    force_only_color
]

for i, (data, label, color) in enumerate(zip(datas, labels, colors)):
    x_axis = data["sweep_vars"][1:]
    y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])[1:]/np.pi
    succ_idxs = data["successes"].flatten()[1:]
    fail_idxs = np.logical_not(succ_idxs)
    plt.plot(x_axis, y_axis, '-', label=label, zorder=i, color=color)
    plt.scatter(x_axis[succ_idxs], y_axis[succ_idxs], color=color, zorder=i)
    plt.scatter(x_axis[fail_idxs], y_axis[fail_idxs], facecolors='none', color=color, zorder=i)
plt.legend(frameon=False)
plt.xlabel(r"$F_\mathrm{offset}$ (N)")
plt.ylabel(r"Progress ($\alpha_{max}/\pi$)")
plt.title("Task progress vs. $F_\mathrm{offset}$")
# plt.title(r"Controller performance as a function of $F_\mathrm{offset}$")
plt.autoscale(enable=True, axis='x', tight=True)
xlims = plt.xlim()
plt.xlim(xlims[0],20)
plt.axhline(1, color='k', linestyle='--')
plt.gcf().savefig("F_offset_sweep.pgf", backend="pgf")
plt.show()

## Impedance

In [ ]:
datas = [
    np.load(base_path + "open_loop__impedance.npz", allow_pickle=True),
    np.load(base_path + "vision_and_force__impedance.npz", allow_pickle=True),
    np.load(base_path + "vision_only__impedance.npz", allow_pickle=True),
    np.load(base_path + "force_only__impedance.npz", allow_pickle=True),
]
labels = [
    "Open loop",
    "Vision and force",
    "Vision only",
    "Force only"
]
colors = [
    open_loop_color,
    vision_and_force_color,
    vision_only_color,
    force_only_color
]
# fig, axs = plt.subplots(len(datas), 1, figsize=(6,4), constrained_layout=True, sharex=True, sharey=True)
fig, axs = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True, sharex=True, sharey=True)

if type(axs) != np.ndarray:
    axs = [axs, axs, axs, axs]

for i, (data, label, color, ax) in enumerate(zip(datas, labels, colors, axs)):
    x_axis = data["sweep_vars"]
    y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])/np.pi
    succ_idxs = data["successes"].flatten()
    fail_idxs = np.logical_not(succ_idxs)
    p = ax.semilogx(4*x_axis, y_axis, '-', label=label, zorder=i, color=color)
    ax.scatter(4*x_axis[succ_idxs], y_axis[succ_idxs], color=p[0].get_color(), zorder=i)
    ax.scatter(4*x_axis[fail_idxs], y_axis[fail_idxs], facecolors='none', color=p[0].get_color(), zorder=i)
plt.axhline(1, color='k', linestyle='--')
plt.legend(frameon=False)
plt.xlabel("Impedance stiffness (N/m)")
plt.ylabel(r"Progress ($\alpha_{max}/\pi$)")
ax.autoscale(enable=True, axis='x', tight=True)
plt.title(r"Task progress vs. translational impedance stiffness ($k_\mathrm{tran}$)")
fig.savefig("impedance_sweep.pgf", backend="pgf")
plt.show()

In [ ]:
plot_manipulator=True
data = np.load(base_path + "vision_and_force__impedance.npz", allow_pickle=True)
sim_args = data["sim_args"][0]

idxs = [-6]
fig, axs, cb = draw_trajs(data, idxs=idxs, plot_manipulator=plot_manipulator, num_times=5,
                          fig_height=3, fig_width=3)#, progs=[0, 0.15, 0.3], max_times=[0.3])
for ax, i in zip(axs, idxs):
    ax.set_title(r"$k_\mathrm{tran} = " + "{:.2f}$".format(4*data["sweep_vars"][i,0]))
plt.suptitle("Hinge trajectory with vision and force feedback")
fig.savefig("impedance_vision_and_force_full_hinge_1.pgf", backend="pgf")
plt.show()

In [ ]:
plot_manipulator=True
data = np.load(base_path + "vision_and_force__impedance.npz", allow_pickle=True)
sim_args = data["sim_args"][0]

idxs = [-1]
fig, axs, cb = draw_trajs(data, idxs=idxs, plot_manipulator=plot_manipulator, num_times=15,
                          fig_height=3, fig_width=3, cb_shrink=1)#, progs=[0, 0.15, 0.3], max_times=[0.3])
for ax, i in zip(axs, idxs):
    ax.set_title(r"$k_\mathrm{tran} = " + "{:.2f}$".format(4*data["sweep_vars"][i,0]))
plt.suptitle("Hinge trajectory with vision and force feedback")
fig.savefig("impedance_vision_and_force_full_hinge_2.pgf", backend="pgf")
plt.show()

In [ ]:
data.files

In [ ]:
idxs = [-1,-2]
colors = ["darkorange"]

data = np.load(base_path + "vision_and_force__impedance.npz", allow_pickle=True)
w_L = data['sim_args'][0]['sim_params'].w_L
h_L = data['sim_args'][0]['sim_params'].h_L
r = data['sim_args'][0]['sim_params'].r

cmap = plt.get_cmap("viridis_r")
for i, color in zip(idxs, colors):
    fig, axs = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True)
    data_ = data['out_data'][i]
    val = data['sweep_vars'][i,0]

    start_idx = data_["any_links_in_contact_idx"]
    plt.plot(data_["horizontal_paper_traces"][-1,start_idx:],
             data_["vertical_paper_traces"][-1,start_idx:],
             color=color, label=r"$k_\textrm{tran} = " + "{:.2f}$".format(val))
    plt.plot(data_["horizontal_paper_traces"][-1,start_idx],
             data_["vertical_paper_traces"][-1,start_idx],
             marker='o', color=color)
    plt.plot(data_["horizontal_paper_traces"][-1,-1],
             data_["vertical_paper_traces"][-1,-1], marker='X', color=color)
    
    start_idx = data_["any_links_in_contact_idx"]
    start_time = data_["times"][start_idx]
    end_time = data_["times"][-1]
    times_ = [start_time, 5.3, 6.6, end_time]
    time_range = end_time - start_time
    for t in times_:
        c = cmap(((t-start_time)/time_range))
        idx = np.argmax(data_["times"] >= t)

        last_link = True
        for x, y, ang in zip(
                data_["horizontal_paper_traces"][::-1,idx],
                data_["vertical_paper_traces"][::-1,idx],
                data_["angle_paper_traces"][::-1,idx]):

            x0 = x - np.cos(ang)*w_L/2
            y0 = y - np.sin(ang)*w_L/2

            rect = plt. Rectangle((x0, y0), w_L, h_L,
                angle=ang*180/np.pi, color=c,alpha=1 if last_link else 0.3)
            plt.gca().add_patch(rect)
            last_link=False

        x = data_["horizontal_manipulator_trace"][idx]
        y = data_["vertical_manipulator_trace"][idx]

        circ = plt.Circle((x, y), r*1.1, color=c, alpha=0.3, snap=False)
        plt.gca().add_patch(circ)
    
plt.legend(frameon=False)
plt.xlabel("$y$ position")
plt.ylabel("$z$ position")
plt.gca().set_aspect("equal")
xlims = plt.xlim()
ylims = plt.ylim()
rect = plt.Rectangle(
    (data_["horizontal_paper_traces"][0,0]-w_L/2,
     data_["vertical_paper_traces"][0,0]-h_L/2),
    10,
    10,
    angle=-90,
    color=(0.2, 0.2, 0.2, 1.0)
)
plt.gca().add_patch(rect)
plt.xlim(xlims)
plt.ylim(ylims)

# fig.savefig("traj_plots.pgf", backend="pgf")
plt.show()

In [ ]:
p_M = np.array([[x, y]]).T

In [ ]:
idxs = [-5, -1]
data = np.load(base_path + "vision_and_force__impedance.npz", allow_pickle=True)


w_L = data['sim_args'][0]['sim_params'].w_L
h_L = data['sim_args'][0]['sim_params'].h_L
r = data['sim_args'][0]['sim_params'].r
color = "darkorange"

cmap = plt.get_cmap("viridis_r")

p_LMT_scalars = []
p_LMT_times = []
inflection_point = 5.3
contact_breack_time = 7.9
vals = []
for i, idx in enumerate(idxs):
    plt.figure(figsize=(6,4))
    data_ = data['out_data'][idx]
    val = data['sweep_vars'][idx,0]
    vals.append(val)

    start_idx = data_["any_links_in_contact_idx"]
    
    # Plot link path
    plt.plot(data_["horizontal_paper_traces"][-1,start_idx:],
             data_["vertical_paper_traces"][-1,start_idx:],
             color=color, label=r"$k_\textrm{tran} = " + "{:.2f}$".format(val))
    # plt.plot(data_["horizontal_paper_traces"][-1,start_idx],
    #          data_["vertical_paper_traces"][-1,start_idx],
    #          marker='X', color=color)
    # plt.plot(data_["horizontal_paper_traces"][-1,-1],
    #          data_["vertical_paper_traces"][-1,-1], marker='X', color=color)
    
    # p_L
    p_L = np.array([[
        data_["horizontal_paper_traces"][-1,start_idx:],
        data_["vertical_paper_traces"][-1,start_idx:]
    ]]).T
    T_hat = np.array([[
        -np.cos(data_["angle_paper_traces"][-1,start_idx:]),
        -np.sin(data_["angle_paper_traces"][-1,start_idx:])
    ]]).T
    p_M = np.array([[
        data_["horizontal_manipulator_trace"][start_idx:],
        data_["vertical_manipulator_trace"][start_idx:]
    ]]).T
    p_LM = p_M - p_L
    p_LMT_scalar = np.matmul(np.transpose(T_hat, [0, 2, 1]), p_LM)
    p_LMT_scalars.append(p_LMT_scalar)
    p_LMT = p_LMT_scalar*T_hat
    p_C = p_LMT + p_L
    if i == 1:
        p_LMT_times.append(data_["times"][start_idx:])
    else:
        p_LMT_times.append(data_["times"][start_idx:])
    

    # Times
    start_idx = data_["any_links_in_contact_idx"]
    start_time = data_["times"][start_idx]
    end_time = data_["times"][-1]
    time_range = end_time - start_time
    times_ = [
        np.linspace(start_time, end_time, 5),
        [start_time, inflection_point, 6.6, end_time],
    ][i]
    plot_contact_point = [
        np.ones(5, dtype=bool),
        [True,       True, True, False]
    ][i]
    for t, plot_contact_point_ in zip(times_, plot_contact_point):
        c = cmap(((t-start_time)/time_range))
        idx = np.argmax(data_["times"] >= t)

        last_link = True
        p_L = None
        T_hat = None
        for x, y, ang in zip(
                data_["horizontal_paper_traces"][::-1,idx],
                data_["vertical_paper_traces"][::-1,idx],
                data_["angle_paper_traces"][::-1,idx]):

            x0 = x - np.cos(ang)*w_L/2
            y0 = y - np.sin(ang)*w_L/2

            if last_link:
                p_L = np.array([[x, y]]).T
                T_hat = np.array([[x0-x, y0 - y]]).T
                T_hat = T_hat/np.linalg.norm(T_hat)

            rect = plt. Rectangle((x0, y0), w_L, h_L,
                angle=ang*180/np.pi, color=c)
            plt.gca().add_patch(rect)
            last_link = False

        x = data_["horizontal_manipulator_trace"][idx]
        y = data_["vertical_manipulator_trace"][idx]

        circ = plt.Circle((x, y), r*1.1, color=c, alpha=0.3, snap=False)
        plt.gca().add_patch(circ)

        p_M = np.array([[x, y]]).T
        p_LM = p_M - p_L
        p_LMT = np.matmul(T_hat.T, p_LM)*T_hat
        p_C = p_LMT + p_L
        if plot_contact_point_:
            circ = plt.Circle((p_C[0,0], p_C[1,0]), 0.005, color=color)
            plt.gca().add_patch(circ)


        # circ = plt.Circle((x, y), r*1.1, color=c, alpha=0.3, snap=False)
        # plt.gca().add_patch(circ)

    # plt.legend(frameon=False)
    plt.xlabel("$y$ position")
    plt.ylabel("$z$ position")
    plt.gca().set_aspect("equal")
    xlims = plt.xlim()
    ylims = plt.ylim()
    rect = plt.Rectangle(
        (data_["horizontal_paper_traces"][0,0]-w_L/2,
         data_["vertical_paper_traces"][0,0]-h_L/2),
        10,
        10,
        angle=-90,
        color=(0.2, 0.2, 0.2, 1.0)
    )
    plt.gca().add_patch(rect)
    plt.xlim(xlims)
    plt.ylim(ylims)
    plt.title(r"$k_\textrm{tran} = " + "{:.2f}$".format(val))
    plt.gcf().savefig("impedance_contact_point_{}.pgf".format(i), backend="pgf")
    plt.show()

# fig.savefig("traj_plots.pgf", backend="pgf")
plt.show()

plt.figure()
fig, ax = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True)
first_run = True
min_x = -np.inf
max_x = np.inf
contact_breack_idx = np.argmax(np.abs(p_LMT_scalars[1]) > w_L/2)
p_LMT_scalars[1] = p_LMT_scalars[1][:contact_breack_idx]
p_LMT_times[1] = p_LMT_times[1][:contact_breack_idx]
colors=['royalblue','darkorange']
for p_LMT_time_, p_LMT_scalar_, val, color in zip(p_LMT_times, p_LMT_scalars, vals, colors):
    def get_normalized_time(t):
        ret = t - p_LMT_time_[0]
        return t / np.max(p_LMT_time_ - p_LMT_time_[0])
    normalized_time = get_normalized_time(p_LMT_time_)
    ax.plot(normalized_time - normalized_time[0], p_LMT_scalar_.flatten(),
            label=r"$k_\textrm{tran} = " + "{:.2f}$".format(val), color=color)
ax.autoscale(enable=True, axis='x', tight=True)
ax.set_xlabel("Normalized time")
ax.set_ylabel(r"Distance from link center of mass in $\hat T$ direction (m)")
plt.axhline(-w_L/2, color='k', linewidth=2, label="Link edge")
plt.axhline(w_L/2, color='k', linewidth=2)
ax.axhline(0, color='k', linestyle='--', label="Link center")
plt.legend(frameon=False, loc=(0.1,0.6))
plt.title("Contact point vs. time")
plt.gcf().savefig("impedance_contact_point.pgf", backend="pgf")
plt.show()

In [ ]:
T_hat.shape

In [ ]:
idx = -1
color = "darkorange"
data = np.load(base_path + "vision_and_force__impedance.npz", allow_pickle=True)

fig, axs = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True)
w_L = data['sim_args'][0]['sim_params'].w_L
h_L = data['sim_args'][0]['sim_params'].h_L
r = data['sim_args'][0]['sim_params'].r

cmap = plt.get_cmap("viridis_r")
data_ = data['out_data'][idx]
val = data['sweep_vars'][idx,0]

start_idx = data_["any_links_in_contact_idx"]
plt.plot(data_["horizontal_paper_traces"][-1,start_idx:],
         data_["vertical_paper_traces"][-1,start_idx:],
         color=color, label=r"$k_\textrm{tran} = " + "{:.2f}$".format(val))
plt.plot(data_["horizontal_paper_traces"][-1,start_idx],
         data_["vertical_paper_traces"][-1,start_idx],
         marker='o', color=color)
plt.plot(data_["horizontal_paper_traces"][-1,-1],
         data_["vertical_paper_traces"][-1,-1], marker='X', color=color)

start_idx = data_["any_links_in_contact_idx"]
start_time = data_["times"][start_idx]
end_time = data_["times"][-1]
times_ = np.linspace(start_time, end_time, 5)
time_range = end_time - start_time
for t in times_:
    c = cmap(((t-start_time)/time_range))
    idx = np.argmax(data_["times"] >= t)

    last_link = True
    p_L = None
    T_hat = None
    for x, y, ang in zip(
            data_["horizontal_paper_traces"][::-1,idx],
            data_["vertical_paper_traces"][::-1,idx],
            data_["angle_paper_traces"][::-1,idx]):

        x0 = x - np.cos(ang)*w_L/2
        y0 = y - np.sin(ang)*w_L/2

        if last_link:
            p_L = np.array([[x, y]]).T
            T_hat = np.array([[x0-x, y0 - y]]).T
            T_hat = T_hat/np.linalg.norm(T_hat)

        rect = plt. Rectangle((x0, y0), w_L, h_L,
            angle=ang*180/np.pi, color=c)
        plt.gca().add_patch(rect)
        last_link = False

    x = data_["horizontal_manipulator_trace"][idx]
    y = data_["vertical_manipulator_trace"][idx]
    # plt.plot([x,x+T_hat[0,0]], [y,y+T_hat[1,0]])

    circ = plt.Circle((x, y), r*1.1, color=c, alpha=0.3, snap=False)
    plt.gca().add_patch(circ)
    
    p_M = np.array([[x, y]]).T
    p_LM = p_M - p_L
    p_LMT = np.matmul(T_hat.T, p_LM)*T_hat
    p_C = p_LMT + p_L
    circ = plt.Circle((p_C[0,0], p_C[1,0]), 0.005, color=color)
    plt.gca().add_patch(circ)


    # circ = plt.Circle((x, y), r*1.1, color=c, alpha=0.3, snap=False)
    # plt.gca().add_patch(circ)
    
plt.legend(frameon=False)
plt.xlabel("$y$ position")
plt.ylabel("$z$ position")
plt.gca().set_aspect("equal")
xlims = plt.xlim()
ylims = plt.ylim()
rect = plt.Rectangle(
    (data_["horizontal_paper_traces"][0,0]-w_L/2,
     data_["vertical_paper_traces"][0,0]-h_L/2),
    10,
    10,
    angle=-90,
    color=(0.2, 0.2, 0.2, 1.0)
)
plt.gca().add_patch(rect)
plt.xlim(xlims)
plt.ylim(ylims)

# fig.savefig("traj_plots.pgf", backend="pgf")
plt.show()

In [ ]:
data_.keys()

In [ ]:
data = np.load(base_path + "vision_and_force__impedance.npz", allow_pickle=True)
idxs = [-4,-1]
fig, axs = plt.subplots(1, len(idxs), figsize=(6,2), constrained_layout=True,sharex=True,sharey=True)
ax.set_ylabel("$z$ position")
end_times = [None, None]
color_map = cm.ScalarMappable(cmap="viridis_r", norm=plt.Normalize(0,np.max(data['sweep_vars'])))
for i, (data_, k_J, ax,end_time) in enumerate(zip(data['out_data'][idxs], data['sweep_vars'][idxs], axs, end_times)):
    c = color_map.to_rgba(k_J)
    start_idx = data_["any_links_in_contact_idx"]
    ax.scatter(data_["horizontal_paper_traces"][-1,start_idx:end_idx:100],
                data_["vertical_paper_traces"][-1,start_idx:end_idx:100],
                c=data_["times"][start_idx:end_idx:100],s=2)
    ax.set_aspect("equal")
    ax.set_xlabel("$y$ position")

# cb = fig.colorbar(color_map)
# cb.set_label("Link stiffness (Nm/rad)")
# plt.gcf().savefig("k_J_plot.pgf", backend="pgf")
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True)

color_map = cm.ScalarMappable(cmap="viridis_r", norm=plt.Normalize(0,np.max(data['sweep_vars'])))
for i, (data_, Fn_noise) in enumerate(zip(data['out_data'], data['sweep_vars'].flatten())):
    print(Fn_noise)
    c = color_map.to_rgba(Fn_noise)
    start_idx = data_["any_links_in_contact_idx"]
    p = plt.plot(data_["horizontal_paper_traces"][-1,start_idx:],
             data_["vertical_paper_traces"][-1,start_idx:], color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,start_idx],
             data_["vertical_paper_traces"][-1,start_idx],
             marker='o', color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,-1],
             data_["vertical_paper_traces"][-1,-1], marker='X', color=c)
plt.xlabel("$y$ position")
plt.ylabel("$z$ position")

cb = fig.colorbar(color_map)
plt.gca().set_aspect("equal")
cb.set_label("$F_N$ noise")
# plt.gcf().savefig("Fn_noise_plot.pgf", backend="pgf")
plt.show()

# Noise
## $F_n$

In [ ]:
data = np.load(base_path + "vision_and_force__Fn_noise.npz", allow_pickle=True)

In [ ]:
num_samples = data["num_samples"]
num_noises = int(data["sweep_vars"].shape[0]/num_samples)
noises = data["sweep_vars"][::num_samples,0]
raw_y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])
raw_succ_idxs = data["successes"].flatten()
raw_fail_idxs = np.logical_not(raw_succ_idxs)
portion_success = []

for i, noise in enumerate(noises):
    data_for_noise = raw_y_axis[i*num_samples:(i+1)*num_samples]
    mean = np.mean(data_for_noise)
    stddev = np.std(data_for_noise, ddof=1)
    plt.plot(np.ones(num_samples)*noise, data_for_noise, ' o', markersize=4, color=vision_and_force_color)
    plt.plot(noise, mean, ' x', markersize=5, color=vision_and_force_color)
    plt.plot([noise, noise], [mean-2*stddev,mean+2*stddev], '-', color=vision_and_force_color)
    
    succ_idxs = raw_succ_idxs[i*num_samples:(i+1)*num_samples]
    fail_idxs = raw_fail_idxs[i*num_samples:(i+1)*num_samples]
    portion_success.append(np.count_nonzero(succ_idxs)/num_samples)
plt.xlabel("Noise")
plt.ylabel("Angle about joint (radians)")
plt.show()

In [ ]:
portion_success       

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True)

color_map = cm.ScalarMappable(cmap="viridis_r", norm=plt.Normalize(0,np.max(data['sweep_vars'])))
for i, (data_, Fn_noise) in enumerate(zip(
    data['out_data'][::data["num_samples"]], data['sweep_vars'].flatten()[::data["num_samples"]])):
    print(Fn_noise)
    c = color_map.to_rgba(Fn_noise)
    start_idx = data_["any_links_in_contact_idx"]
    p = plt.plot(data_["horizontal_paper_traces"][-1,start_idx:],
             data_["vertical_paper_traces"][-1,start_idx:], color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,start_idx],
             data_["vertical_paper_traces"][-1,start_idx],
             marker='o', color=c)
    plt.plot(data_["horizontal_paper_traces"][-1,-1],
             data_["vertical_paper_traces"][-1,-1], marker='X', color=c)
plt.xlabel("$y$ position")
plt.ylabel("$z$ position")

cb = fig.colorbar(color_map)
plt.gca().set_aspect("equal")
cb.set_label("$F_N$ noise")
plt.gcf().savefig("Fn_noise_plot.pgf", backend="pgf")
plt.show()

In [ ]:
plt.figure()
plt.plot(data["out_data"][0]["times"], data["out_data"][0]["max_F_ONs"])
plt.show()

## Link noise

In [ ]:
datas = [
    np.load(base_path + "vision_and_force__link_noise.npz", allow_pickle=True),
    np.load(base_path + "vision_only__link_noise.npz", allow_pickle=True),
]
labels = ["Measured $F_n$", "Constant"]
colors = [vision_and_force_color, vision_only_color]
fig, axs = plt.subplots(len(datas), 1, figsize=(6,4), constrained_layout=True, sharex=True, sharey=True)
if len(datas) == 1:
    axs = [axs]
portion_successes = []

for data, label, color, ax in zip(datas, labels, colors, axs):
    num_samples = data["num_samples"]
    num_noises = int(data["sweep_vars"].shape[0]/num_samples)
    noises = data["sweep_vars"][::num_samples,0]
    raw_y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])
    raw_succ_idxs = data["successes"].flatten()
    raw_fail_idxs = np.logical_not(raw_succ_idxs)
    means = []
    portion_success = []
    
    for i, noise in enumerate(noises):
        # print(noise)
        data_for_noise = raw_y_axis[i*num_samples:(i+1)*num_samples]
        mean = np.mean(data_for_noise)
        means.append(mean)
        stddev = np.std(data_for_noise, ddof=1)
        ax.semilogx(
            [noise, noise], np.array([mean-2*stddev,mean+2*stddev])/success_angle, '-', color=color)
        
        x_axis = np.ones(num_samples)*noise
        y_axis = data_for_noise
        succ_idxs = raw_succ_idxs[i*num_samples:(i+1)*num_samples]
        fail_idxs = raw_fail_idxs[i*num_samples:(i+1)*num_samples]
        # print(succ_idxs)
        # print(np.count_nonzero(succ_idxs))
        
        portion_success.append(np.count_nonzero(succ_idxs)/num_samples)
        
        ax.scatter(x_axis[succ_idxs], y_axis[succ_idxs]/success_angle, color=color, zorder=i, alpha=0.1)
        ax.scatter(x_axis[fail_idxs], y_axis[fail_idxs]/success_angle, facecolors='none',
                   color=color, zorder=i, alpha=0.5)
    portion_successes.append(portion_success)
        
    ax.set_title(label)
    ax.semilogx(noises, np.array(means)/success_angle, '--x', markersize=10, color=color)
ax.set_xlabel("Link noise standard deviation")
ax.set_ylabel("Angle about joint (radians)")
plt.ylim(0,1.2)
plt.show()

In [ ]:
datas = [
    np.load(base_path + "vision_and_force__link_noise.npz", allow_pickle=True),
    np.load(base_path + "vision_only__link_noise.npz", allow_pickle=True),
]
labels = ["Measured $F_n$", "Constant"]
colors = [vision_and_force_color, vision_only_color]
fig, ax = plt.subplots(1, 1, figsize=(6,4), constrained_layout=True, sharex=True, sharey=True)
portion_successes = []

for data, label, color in zip(datas, labels, colors):
    num_samples = data["num_samples"]
    num_noises = int(data["sweep_vars"].shape[0]/num_samples)
    noises = data["sweep_vars"][::num_samples,0]
    # raw_y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])
    raw_succ_idxs = data["successes"].flatten()
    raw_fail_idxs = np.logical_not(raw_succ_idxs)
    means = []
    portion_success = []
    
    for i in range(len(noises)):
        succ_idxs = raw_succ_idxs[i*num_samples:(i+1)*num_samples]
        fail_idxs = raw_fail_idxs[i*num_samples:(i+1)*num_samples]
        
        portion_success.append(np.count_nonzero(succ_idxs)/num_samples)
    portion_successes.append(portion_success)
    ax.semilogx(noises, portion_success, color=color, zorder=i)
        
    ax.set_title(label)
ax.set_xlabel("Link noise standard deviation")
ax.set_ylabel("Angle about joint (radians)")
plt.ylim(0,1.2)
plt.show()

In [ ]:
datas = [
    np.load(base_path + "vision_and_force__Fn_noise.npz", allow_pickle=True),
    np.load(base_path + "force_only__force_noise.npz", allow_pickle=True),
]
labels = [
    "Vision and force",
    "Force only"
]
colors = [
    vision_and_force_color,
    force_only_color
]
fig, axs = plt.subplots(len(datas), 1, figsize=(6,4), constrained_layout=True, sharex=True, sharey=True)
if len(datas) == 1:
    axs = [axs]
portion_successes = []

for data, label, color, ax in zip(datas, labels, colors, axs):
    print(color)
    num_samples = data["num_samples"]
    num_noises = int(data["sweep_vars"].shape[0]/num_samples)
    noises = data["sweep_vars"][::num_samples,0]
    raw_y_axis = np.array([val["max_overall_theta"] for val in data["out_data"]])
    raw_succ_idxs = data["successes"].flatten()
    raw_fail_idxs = np.logical_not(raw_succ_idxs)
    means = []
    portion_success = []
    
    for i, noise in enumerate(noises):
        print(noise)
        data_for_noise = raw_y_axis[i*num_samples:(i+1)*num_samples]
        mean = np.mean(data_for_noise)
        means.append(mean)
        stddev = np.std(data_for_noise, ddof=1)
        ax.semilogx(
            [noise, noise], np.array([mean-2*stddev,mean+2*stddev])/success_angle, '-', color=color)
        
        x_axis = np.ones(num_samples)*noise
        y_axis = data_for_noise
        succ_idxs = raw_succ_idxs[i*num_samples:(i+1)*num_samples]
        fail_idxs = raw_fail_idxs[i*num_samples:(i+1)*num_samples]
        # print(succ_idxs)
        # print(np.count_nonzero(succ_idxs))
        
        print(np.count_nonzero(succ_idxs))
        portion_success.append(np.count_nonzero(succ_idxs)/num_samples)
        if np.any(succ_idxs):
            ax.scatter(x_axis[succ_idxs],
                       y_axis[succ_idxs]/success_angle, color=color, zorder=i, alpha=0.1)
        if np.any(fail_idxs):
            ax.scatter(x_axis[fail_idxs],
                   y_axis[fail_idxs]/success_angle, facecolors='none',
                   color=color, zorder=i, alpha=0.5)
    portion_successes.append(portion_success)
        
    ax.set_title(label)
    ax.semilogx(noises, np.array(means)/success_angle, '--x', markersize=10, color=color)
ax.set_xlabel("Link noise standard deviation")
ax.set_ylabel("Angle about joint (radians)")
plt.ylim(0,1.2)
plt.show()

In [ ]:
plt.figure()
for portion_success_ in portion_successes:
    plt.plot(portion_success_)

In [ ]:
data["exit_messages"]